In [2]:
!ls

drive  sample_data


In [35]:
!pip install -r requirements_cpu.txt

     |████████████████████████████████| 163kB 6.1MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 3.0MB 12.4MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 3.3MB 40.9MB/s 
     |████████████████████████████████| 122kB 60.1MB/s 
     |████████████████████████████████| 788kB 59.0MB/s 
     |████████████████████████████████| 1.1MB 61.7MB/s 
     |████████████████████████████████| 296kB 57.5MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 7.8MB 25.5MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 12.4MB 245kB/s 
     |████████████████████████████████| 174kB 59.3MB/s 
     |████████████████████████████████| 9.7MB 19.3MB/s 
     |████████████████████████████████| 20.2MB 1.2MB/s 
     |████████████████████████████████| 28.2MB 79kB/s 
  

In [5]:
!pip freeze

absl-py==0.9.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appnope==0.1.0
argon2-cffi==20.1.0
asgiref==3.2.10
astor==0.8.1
astropy==4.0.1.post1
astunparse==1.6.3
atari-py==0.2.6
atomicwrites==1.4.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.8.0
backcall==0.1.0
beautifulsoup4==4.6.3
bleach==3.1.5
blis==0.4.1
bokeh==2.1.1
boto==2.49.0
boto3==1.14.37
botocore==1.17.37
Bottleneck==1.3.2
branca==0.4.1
bs4==0.0.1
CacheControl==0.12.6
cachetools==4.1.0
catalogue==1.0.0
certifi==2020.4.5.1
cffi==1.14.1
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.4.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.1
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda101==7.4.0
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.3
Cython==0.29.21
daft==0.0.4
dask==2.12.0
dataclasses==0.7
datascience==0.10.6
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.2
distributed==1.25.3
Django==3.1

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls

drive  sample_data


In [9]:
%cd '/content/drive/My Drive/road-extraction-service/'

/content/drive/My Drive/road-extraction-service


In [10]:
! ls data/testing/input/ | wc -l

52


In [11]:
import glob

import numpy as np
import functools
import random
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from tensorflow.keras import losses
import tensorflow.keras.backend as K
import segmentation_models as sm

from cb.tbi_cb import TensorBoardImage
from cb.snapshot_cb_builder import SnapshotCallbackBuilder
from cb.sgdr_lr_scheduler import SGDRScheduler

Segmentation Models: using `tf.keras` framework.


In [37]:
img_dir = "data/training/input"
label_dir = "data/training/output"
train_files = glob.glob(img_dir + "/*.png")
train_label_files = []
for x in train_files:
    train_label_files.append(x.replace("input", "output"))

print(train_label_files[0])
print(train_files[0])

x_train_filenames, x_val_filenames, y_train_filenames, y_val_filenames = train_test_split(train_files,
                                                                                          train_label_files,
                                                                                          test_size=0.1,
                                                                                          random_state=42)

num_train_examples = len(x_train_filenames)
num_val_examples = len(x_val_filenames)

print("Number of training examples: {}".format(num_train_examples))
print("Number of validation examples: {}".format(num_val_examples))

data/training/output/img-121a.png
data/training/input/img-121a.png
Number of training examples: 2718
Number of validation examples: 303


In [38]:
img_shape = (512, 512, 3)
batch_size = 4
n_classes = 1
epochs = 70
BACKBONE = 'efficientnetb4'

preprocess_input = sm.get_preprocessing(BACKBONE)

In [39]:
def _process_pathnames(fname, label_path):
    img_str = tf.io.read_file(fname)
    img = tf.image.decode_png(img_str, channels=3)

    label_img_str = tf.io.read_file(label_path)
    label_img = tf.image.decode_png(label_img_str)

    label_img = label_img[:, :, 0]
    label_img = tf.expand_dims(label_img, axis=-1)
    return img, label_img

def shift_img(output_img, label_img, width_shift_range, height_shift_range):
    if width_shift_range or height_shift_range:
        if width_shift_range:
            width_shift_range = tf.random.uniform([],
                                                  -width_shift_range * img_shape[1],
                                                  width_shift_range * img_shape[1])
        if height_shift_range:
            height_shift_range = tf.random.uniform([],
                                                   -height_shift_range * img_shape[0],
                                                   height_shift_range * img_shape[0])
        # Translate both
        output_img = tfa.image.translate(output_img,
                                         [width_shift_range, height_shift_range])
        label_img = tfa.image.translate(label_img,
                                        [width_shift_range, height_shift_range])
    return output_img, label_img


def flip_img(horizontal_flip, tr_img, label_img):
    if horizontal_flip:
        flip_prob = tf.random.uniform([], 0.0, 1.0)
        tr_img, label_img = tf.cond(tf.math.less(flip_prob, 0.5),
                                    lambda: (tf.image.flip_left_right(tr_img), tf.image.flip_left_right(label_img)),
                                    lambda: (tr_img, label_img))
    return tr_img, label_img


def _augment(img,
             label_img,
             resize=None,  # Resize the image to some size e.g. [256, 256]
             scale=1,  # Scale image e.g. 1 / 255.
             hue_delta=0,  # Adjust the hue of an RGB image by random factor
             horizontal_flip=False,  # Random left right flip,
             width_shift_range=0,  # Randomly translate the image horizontally
             height_shift_range=0):  # Randomly translate the image vertically
    if resize is not None:
        # Resize both images
        label_img = tf.image.resize(label_img, resize)
        img = tf.image.resize(img, resize)

    brightness_prob = tf.random.uniform([], 0.0, 1.0)
    if tf.math.less(brightness_prob, 0.5):
        img = tf.image.adjust_brightness(img, 0.2)

    if hue_delta:
        img = tf.image.random_hue(img, hue_delta)

    img, label_img = flip_img(horizontal_flip, img, label_img)
    # img, label_img = shift_img(img, label_img, width_shift_range, height_shift_range)
    label_img = tf.cast(label_img, dtype=tf.float32) * scale
    img = tf.cast(img, dtype=tf.float32) * scale
    return img, label_img


def _tb_augment(img, label_img):
    label_img = tf.image.resize(label_img, [img_shape[0], img_shape[1]])
    img = tf.image.resize(img, [img_shape[0], img_shape[1]])

    label_img = tf.cast(label_img, dtype=tf.float32) * (1 / 255.)
    img = tf.cast(img, dtype=tf.float32) * (1 / 255.)

    return img, label_img


def get_baseline_dataset(filenames,
                         labels,
                         preproc_fn=functools.partial(_augment),
                         threads=6,
                         batch_size=batch_size,
                         shuffle=False):
    num_x = len(filenames)
    # Create a dataset from the filenames and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    # Map our preprocessing function to every element in our dataset, taking
    # advantage of multithreading
    dataset = dataset.map(_process_pathnames, num_parallel_calls=threads)
    # print(dataset)
    if preproc_fn.keywords is not None and 'resize' not in preproc_fn.keywords:
        assert batch_size == 1, "Batching images must be of the same size"

    dataset = dataset.map(preproc_fn, num_parallel_calls=threads)

    if shuffle:
        dataset = dataset.shuffle(num_x)

    # It's necessary to repeat our data for all epochs
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat()
    # dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    # dataset = dataset.repeat().batch(batch_size)
    return dataset


def get_tb_dataset(fnames,
                   lnames,
                   preproc_fn=functools.partial(_tb_augment),
                   threads=6,
                   batch_size=1,
                   shuffle=True):
    filenames, labels = zip(*random.sample(list(zip(fnames, lnames)), 30))
    filenames = list(filenames)
    labels = list(labels)
    num_x = len(filenames)
    # Create a dataset from the filenames and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    dataset = dataset.map(_process_pathnames, num_parallel_calls=threads)
    # print(dataset)
    if preproc_fn.keywords is not None and 'resize' not in preproc_fn.keywords:
        assert batch_size == 1, "Batching images must be of the same size"

    dataset = dataset.map(preproc_fn, num_parallel_calls=threads)

    if shuffle:
        dataset = dataset.shuffle(num_x)

    # It's necessary to repeat our data for all epochs
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    # dataset = dataset.repeat().batch(batch_size)

    return dataset

In [40]:
tr_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
    'hue_delta': 0.1,
    'horizontal_flip': True,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1
}
tr_preprocessing_fn = functools.partial(_augment, **tr_cfg)

val_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
}
val_preprocessing_fn = functools.partial(_augment, **val_cfg)

train_ds = get_baseline_dataset(x_train_filenames,
                                y_train_filenames,
                                preproc_fn=tr_preprocessing_fn,
                                batch_size=batch_size)
val_ds = get_baseline_dataset(x_val_filenames,
                              y_val_filenames,
                              preproc_fn=val_preprocessing_fn,
                              batch_size=batch_size)
tb_ds = get_tb_dataset(x_val_filenames, y_val_filenames)

In [41]:
# LOSSES

def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


# https://www.programmersought.com/article/60001511310/
def binary_focal_loss(gamma=2, alpha=0.25):
    """
    Binary form of focal loss.
         Focal loss for binary classification problems

    focal_loss(p_t) = -alpha_t * (1 - p_t)**gamma * log(p_t)
        where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        y_true shape need be (None,1)
        y_pred need be compute after sigmoid
        """
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (K.ones_like(y_true) - y_true) * (1 - alpha)

        p_t = y_true * y_pred + (K.ones_like(y_true) - y_true) * (K.ones_like(y_true) - y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true) - p_t), gamma) * K.log(p_t)
        return K.mean(focal_loss)

    return binary_focal_loss_fixed

In [42]:
steps_per_epoch = int(np.ceil(num_train_examples / float(batch_size)))
validation_steps = int(np.ceil(num_val_examples / float(batch_size)))

# Sets up a timestamped log directory.
logdir = "logs/train_data/"

# Callbacks
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0, write_graph=True, write_images=True)
tbi_callback = TensorBoardImage(logdir, data_set=tb_ds)

sgdr_lr = SGDRScheduler(min_lr=1e-5,
                        max_lr=1e-2,
                        steps_per_epoch=steps_per_epoch,
                        swa_path='swa_weights/model_swa_{}.hdf5',
                        tb_log_dir=logdir,
                        lr_decay=0.9,
                        cycle_length=5,
                        mult_factor=1.5)

snapshot = SnapshotCallbackBuilder(sgdr_lr=sgdr_lr, nb_epochs=epochs, nb_snapshots=1, init_lr=1e-3)
snapshot_callbacks = snapshot.get_callbacks()


In [43]:
# DEFINE MODEL

model = sm.Unet(BACKBONE, classes=1, activation='sigmoid')


# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
#dice_loss = dice_coef_loss()
#focal_loss = binary_focal_loss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), sm.metrics.Precision(threshold=0.5),
          sm.metrics.Recall(threshold=0.5), dice_loss]

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=metrics)

model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 4 1296        input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 4 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 4 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, validation_steps=validation_steps, epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    callbacks=[tbCallBack, snapshot_callbacks, tbi_callback])
final_model_path = "model_train_end.hdf5"
model.save(final_model_path)
print(history)

Epoch 1/70
680/680 [==============================] - ETA: 0s - loss: 0.5069 - iou_score: 0.4920 - f1-score: 0.6527 - precision: 0.6628 - recall: 0.6647 - dice_loss: 0.3777
Epoch 00001: val_dice_loss improved from inf to 0.41831, saving model to ./model.hdf5
680/680 [==============================] - 810s 1s/step - loss: 0.5069 - iou_score: 0.4920 - f1-score: 0.6527 - precision: 0.6628 - recall: 0.6647 - dice_loss: 0.3777 - val_loss: 0.5716 - val_iou_score: 0.4387 - val_f1-score: 0.6054 - val_precision: 0.8120 - val_recall: 0.4912 - val_dice_loss: 0.4183
Epoch 2/70
680/680 [==============================] - ETA: 0s - loss: 0.4236 - iou_score: 0.5556 - f1-score: 0.7112 - precision: 0.7205 - recall: 0.7115 - dice_loss: 0.3145
Epoch 00002: val_dice_loss improved from 0.41831 to 0.29737, saving model to ./model.hdf5
680/680 [==============================] - 543s 799ms/step - loss: 0.4236 - iou_score: 0.5556 - f1-score: 0.7112 - precision: 0.7205 - recall: 0.7115 - dice_loss: 0.3145 - val_